# Health Research Regulator

Researchers must apply to this authority before they can do research on healthcare data. The regulator then issues them a credential that data owners (Hospital's) can verify. Providing hospitals an easy way to authenticate access to sensitive but useful healthcare data.

In [1]:
%autoawait
import time
import asyncio
import nest_asyncio
from termcolor import colored, cprint
nest_asyncio.apply()
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8032
ADMIN_URL = "http://regulator-agent:8031"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


## Generate a new DID

In [2]:
# generate new DID
response = await agent_controller.wallet.create_did()

did_object = response['result']
print("New DID", did_object)

New DID {'did': 'CFiSw2SSYoL9UZvkVV7GWP', 'verkey': '78kbXzRKsBbqJoP8pmEdatn5pLS4Grc4qtP7PxqWHGuG', 'posture': 'wallet_only'}


In [3]:
## Write DID to Sovrin StagingNet

In [4]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())
print(r.status_code)

{'statusCode': 200, 'headers': {'Access-Control-Allow-Origin': '*'}, 'body': '{"statusCode": 200, "CFiSw2SSYoL9UZvkVV7GWP": {"status": "Success", "statusCode": 200, "reason": "Successfully wrote NYM identified by CFiSw2SSYoL9UZvkVV7GWP to the ledger with role ENDORSER"}}'}
200


## Accept Transaction Author Agreement

In [5]:
response = await agent_controller.ledger.get_taa()
TAA_text = response['result']['taa_record']['text']
TAA_version = response['result']['taa_record']['version']
TAA_digest = response['result']['taa_record']['digest']


taa_record = {
            "version": TAA_version,
            "mechanism": "service_agreement",
            'text': TAA_text
}

response = await agent_controller.ledger.accept_taa(taa_record)
## Will return {} if successful
print(response)

{}


## Assign public DID

In [6]:
response = await agent_controller.wallet.assign_public_did(did_object["did"])
print(response)

{'result': {'did': 'CFiSw2SSYoL9UZvkVV7GWP', 'verkey': '78kbXzRKsBbqJoP8pmEdatn5pLS4Grc4qtP7PxqWHGuG', 'posture': 'public'}}


### Optional Write Schema

You will need to change the proof request configuration for hosptials if you do this.

In [7]:

# Or feel free to define your own schema

# Define you schema name - must be unique on the ledger
# schema_name = "Authorised Research"
# # Can version the schema if you wish to update it
# schema_version = "0.0.1"
# # Define any list of attributes you wish to include in your schema
# attributes = ["scope"]

# response = await agent_controller.schema.write_schema(schema_name, attributes, schema_version)
# schema_id = response["schema_id"]
# print(schema_id)

In [8]:
schema_id = "FWyJ74b38bCF1QbYF7tWMM:2:Authorised Research:0.0.1"
response = await agent_controller.definitions.write_cred_def(schema_id)
cred_def_id = response["credential_definition_id"]

## Register Listeners

These handlers ensure that whoever connects to this agent automatically get's issued a credential for the scope of diabetes. Clearly in the real world a more thorough application and research regulation process would be enforced here.

In [9]:
loop = asyncio.get_event_loop()
loop.create_task(agent_controller.listen_webhooks())

def cred_handler(payload):
    print("Handle Credentials")
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    attributes = payload['credential_proposal_dict']['credential_proposal']['attributes']
    print(f"Credential exchange {exchange_id}, role: {role}, state: {state}")
    print(f"Offering: {attributes}")
    
cred_listener = {
    "topic": "issue_credential",
    "handler": cred_handler
}

def connections_handler(payload):
    global STATE
    connection_id = payload["connection_id"]
    print("Connection message", payload, connection_id)
    STATE = payload['state']
    loop = asyncio.get_event_loop()
    if STATE == 'request':
        print("ACCEPTING REQUEST")

        loop.run_until_complete(agent_controller.connections.accept_request(connection_id))
    elif STATE == 'response':
        print("SEND PING")
        time.sleep(2)
        loop.run_until_complete(agent_controller.messaging.trust_ping(connection_id, "hey"))
    elif STATE == 'active':
#         print('Connection {0} changed state to active'.format(connection_id))
        print(colored("Connection {0} changed state to active".format(connection_id), "red", attrs=["bold"]))
        credential_attributes = [{"name": "scope", "value": "diabetes"}]
        ## Offer credential
        time.sleep(1)
        loop.run_until_complete(agent_controller.issuer.send_credential(connection_id, schema_id, cred_def_id, credential_attributes, trace=False))

connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

agent_controller.register_listeners([cred_listener,connection_listener], defaults=True)

In [10]:
invite = await agent_controller.connections.create_invitation(multi_use="true")
print(invite['invitation'])

Connection message {'invitation_mode': 'multi', 'initiator': 'self', 'connection_id': '75f0430e-c639-4f28-93e1-82c57bbf6fde', 'accept': 'manual', 'routing_state': 'none', 'invitation_key': '5GbdLanjtXFE5anxYHwTAy9iAZm1bgLoxpz1va3xPmfC', 'created_at': '2021-02-01 17:18:42.603512Z', 'updated_at': '2021-02-01 17:18:42.603512Z', 'state': 'invitation'} 75f0430e-c639-4f28-93e1-82c57bbf6fde
{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '48db8bb7-0543-4102-a391-1c4f1ed7dfde', 'label': 'NHS Health Research Authority', 'recipientKeys': ['5GbdLanjtXFE5anxYHwTAy9iAZm1bgLoxpz1va3xPmfC'], 'serviceEndpoint': 'http://172.17.0.1:8030'}
Connection message {'my_did': 'WUdzYmrPyPw4h4c2Y8PCcd', 'invitation_mode': 'once', 'initiator': 'multiuse', 'connection_id': 'e02967e0-0f22-4727-b8f9-7ff147a8bb51', 'accept': 'manual', 'routing_state': 'none', 'invitation_key': '5GbdLanjtXFE5anxYHwTAy9iAZm1bgLoxpz1va3xPmfC', 'created_at': '2021-02-01 17:18:56.864158Z', 'updated_at': '

In [ ]:
await agent_controller.terminate()